# Model interpretation

If a machine learning model performs well, why do not we just trust the model and ignore why it made a certain decision? “The problem is that a single metric, such as classification accuracy, is an incomplete description of most real-world tasks.”

Let us dive deeper into the reasons why interpretability is so important. When it comes to predictive modeling, you have to make a trade-off: Do you just want to know what is predicted? For example, the probability that a customer will churn or how effective some drug will be for a patient. Or do you want to know why the prediction was made and possibly pay for the interpretability with a drop in predictive performance? In some cases, you do not care why a decision was made, it is enough to know that the predictive performance on a test dataset was good. But in other cases, knowing the ‘why’ can help you learn more about the problem, the data and the reason why a model might fail. Some models may not require explanations because they are used in a low-risk environment, meaning a mistake will not have serious consequences, (e.g. a movie recommender system) or the method has already been extensively studied and evaluated (e.g. optical character recognition). The need for interpretability arises from an incompleteness in problem formalization, which means that for certain problems or tasks it is not enough to get the prediction (the what). The model must also explain how it came to the prediction (the why), because a correct prediction only partially solves your original problem. The following reasons drive the demand for interpretability and explanations.

If you can ensure that the machine learning model can explain decisions, you can also check the following traits more easily:
* **Fairness**: Ensuring that predictions are unbiased and do not implicitly or explicitly discriminate against protected groups. An interpretable model can tell you why it has decided that a certain person should not get a loan, and it becomes easier for a human to judge whether the decision is based on a learned demographic (e.g. racial) bias.
* **Privacy**: Ensuring that sensitive information in the data is protected.
* **Reliability or Robustness**: Ensuring that small changes in the input do not lead to large changes in the prediction.
* **Causality**: Check that only causal relationships are picked up.
* **Trust**: It is easier for humans to trust a system that explains its decisions compared to a black box.

## Linear (Logistic) Regression

Linear models can be used to model the dependence of a regression target y on some features x. The learned relationships are linear and can be written for a single instance i as follows:
$$y = \beta_0 + \beta_1*x_1 + ... + \beta_n*x_n + e$$

A solution for classification is logistic regression. Instead of fitting a straight line or hyperplane, the logistic regression model uses the logistic function to squeeze the output of a linear equation between 0 and 1. The logistic function is defined as:
$$ logistic(\eta) = \frac{1}{1 + exp(-\eta)}$$

Interpetation of odds (probability of event divided by probability of no event):
$$ \frac{odds_{x_j+1}}{odds_{x_j}} = exp(\beta_j) $$

A change in a feature by one unit changes the odds ratio (multiplicative) by a factor of $exp(\beta_j)$. We could also interpret it this way: а change in $x_j$ by one unit increases the log odds ratio by the value of the corresponding weight.


In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv("../data/train_titanic.csv")
test = pd.read_csv("../data/test_titanic.csv")

In [ ]:
TARGET = "Survived"
y = train[TARGET].values
train = train.drop(TARGET, axis = 1)
features = list(train.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg = LogisticRegression(C=0.1)
log_reg.fit(train, y)

In [ ]:
[f"{feature}: {coef:.2f}" for coef, feature in zip(log_reg.coef_[0], features)]

In [ ]:
log_reg.intercept_

In [ ]:
import numpy as np

In [ ]:
1/(1+np.exp(sum(train.iloc[0]*log_reg.coef_[0]) + log_reg.intercept_))

In [ ]:
log_reg.predict_proba([train.iloc[0]])

In [ ]:
# age odds
np.exp(-0.13)

In [ ]:
temp = train.iloc[0].copy()
temp["Age"] += 1

In [ ]:
1/(1+np.exp(sum(train.iloc[0]*log_reg.coef_[0]) + log_reg.intercept_)*np.exp(log_reg.coef_[0][2]))

In [ ]:
log_reg.predict_proba([temp])

[How to get logistic regression coefficients and their std and p-values.](https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression)

But we can't understand what feature is the most important. First of all, we need to scale all features, retrain a model and absolute coefficients values will describe feature importance.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train)

In [ ]:
train_scaled

In [ ]:
log_reg = LogisticRegression(C=0.1)
log_reg.fit(train_scaled, y)

In [ ]:
[f"{feature}: {coef:.2f}" for coef, feature in zip(log_reg.coef_[0], features)]

## Decision tree

The interpretation is simple: Starting from the root node, you go to the next nodes and the edges tell you which subsets you are looking at. Once you reach the leaf node, the node tells you the predicted outcome. All the edges are connected by ‘AND’.

Template: If feature x is smaller/bigger than threshold AND … then the predicted outcome is the mean value of y of the instances in that node.

**Feature importance**

The overall importance of a feature in a decision tree can be computed in the following way: Go through all the splits for which the feature was used and measure how much it has reduced the variance or [Gini index](https://blog.quantinsti.com/gini-index/) compared to the parent node. The sum of all importances is scaled to 100. This means that each importance can be interpreted as share of the overall model importance.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
dt = DecisionTreeClassifier(max_depth=5)
dt.fit(train, y)

In [ ]:
from subprocess import call
from IPython.display import Image

In [ ]:
def visualize_tree(tree, features):
    export_graphviz(tree, out_file='tree.dot', 
                feature_names = features,
                class_names = ["Not survived", "Survived"],
                rounded = True, proportion = False, 
                precision = 2, filled = True)
    call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

In [ ]:
visualize_tree(dt, features)
Image(filename = 'tree.png')

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=4, n_jobs=-1, )
rf.fit(train, y)

In [ ]:
visualize_tree(rf.estimators_[0], features)
Image(filename = 'tree.png')

In [ ]:
visualize_tree(rf.estimators_[1], features)
Image(filename = 'tree.png')

In [ ]:
sorted([(feature, round(score,2)) for score, feature in zip(rf.feature_importances_, features)], key=lambda x: x[1])

## Xgboost

In [ ]:
import xgboost as xgb

In [ ]:
parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.1,
    "max_depth": 4,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "verbosity": 0,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": "auc"
}

In [ ]:
train_xgb = xgb.DMatrix(train, y)
xgb_model = xgb.train(parameters, train_xgb, num_boost_round=10, verbose_eval=False)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [70, 50]

In [ ]:
xgb.plot_tree(xgb_model, num_trees=0)
plt.show()

In [ ]:
xgb.plot_tree(xgb_model, num_trees=4)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
xgb.plot_importance(xgb_model, );

Don't forget about [xgbfir](https://github.com/limexp/xgbfir). 

## Lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,
    
    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 15
}


In [ ]:
train_lgb = lgb.Dataset(train, y)
lgb_model = lgb.train(parameters, train_lgb, num_boost_round=10, verbose_eval=False)

In [ ]:
plt.rcParams['figure.figsize'] = [70, 50]
lgb.plot_tree(lgb_model, tree_index=5);

In [ ]:
plt.rcParams['figure.figsize'] = [5, 5]
lgb.plot_importance(lgb_model);

## Catboost

In [ ]:
import catboost as ctb

In [ ]:
parameters = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 1000,
    "learning_rate": 0.03,
    "random_seed": 42,
    "od_wait": 30,
    "od_type": "Iter",
    "thread_count": 10
}

In [ ]:
ctb_data = ctb.Pool(train, y)
ctb_model = ctb.train(ctb_data, parameters, iterations=10, verbose=False)

In [ ]:
ctb_model.plot_tree(0, ctb_data)

Select features and visualize how predictions will be changed.

In [ ]:
prediction_variations = ctb_model.plot_predictions(test.iloc[:2], [0,1])

## Model analysis

![image](https://miro.medium.com/max/2560/1*XrQOPzH1aaaUQbubhnDURA.png)

### Feature Importance

* Remove unnecessary features to simplify the model and reduce training/prediction time
* Get the most influential feature for your target value and manipulate them for business gains (eg: healthcare providers want to identify what factors are driving each patient’s risk of some disease so they can directly address those risk factors with targeted medicines)

Different feature importance metrics:
- PredictionValuesChange: calculate score for every feature.
- LossFunctionChange: calculate score for every feature by loss (recommended to use for ranking metrics)
- FeatureImportance 
- ShapValues: calculate SHAP Values for every object.
- Interaction: calculate pairwise score between every feature (similar to xgbfir)
- PredictionDiff: calculate most important features explaining difference in predictions for a pair of documents.

In [ ]:
sorted([(feature, round(score,2)) for score, feature in zip(ctb_model.feature_importances_, features)], key=lambda x: x[1])

In [ ]:
ctb_model.get_feature_importance(ctb_data, "Interaction")[:5]

### Object Importance

* Remove the most useless training objects from the training data
* Prioritize a batch of new objects for labeling based on which ones are expected to be the most “helpful”, akin to active learning

With this functionality, you can calculate the impact of each object on the optimized metric for test data. 
* Positive values reflect that the optimized metric increases
* Negative values reflect that the optimized metric decreases. 

This method is an implementation of the approach described in [this paper](https://arxiv.org/abs/1802.06640). 

In [ ]:
result = ctb_model.get_object_importance(ctb_data, ctb_data, update_method="AllPoints")

### Plots per feature

With this feature, we will be able to visualize how the algorithm is splitting the data for each feature and look at feature-specific statistics. More specifically we will be able to see:

* Average target (label) value in the bucket.
* Average prediction in the bucket.
* Number of objects in the bucket.
* Average predictions on varying values of the feature.
    
    To calculate it, the value of the feature is successively changed to fall into every bucket for every input object. The value for a bucket on the graph is calculated as the average for all objects when their feature values are changed to fall into this bucket.
    
*This plot will give us information like how uniform our splitting is (we don’t want all the objects to go in one bin), whether our predictions are close to target (blue and orange line), the red line will tell us how sensitive our predictions are wrt a feature.*

In [ ]:
features_stats = ctb_model.calc_feature_statistics(ctb_data)

## Shapley Values

A prediction can be explained by assuming that each feature value of the instance is a “player” in a game where the prediction is the payout. Shapley values – a method from coalitional game theory – tells us how to fairly distribute the “payout” among the features.


**The Shapley value is the average marginal contribution of a feature value across all possible coalitions.**

The feature contributions must add up to the difference of prediction for $x$ and the average.

$$\sum_{j=1}^{p}\phi_j = f(x) - E_X(f(X))$$

More details [here](https://christophm.github.io/interpretable-ml-book/shapley.html#general-idea).

**Intuition**

An intuitive way to understand the Shapley value is the following illustration: The feature values enter a room in random order. All feature values in the room participate in the game (= contribute to the prediction). The Shapley value of a feature value is the average change in the prediction that the coalition already in the room receives when the feature value joins them.

**The interpretation of the Shapley value is: Given the current set of feature values, the contribution of a feature value to the difference between the actual prediction and the mean prediction is the estimated Shapley value.**


## Shap

SHAP (SHapley Additive exPlanations) is a method to explain individual predictions. SHAP is based on the game theoretically optimal Shapley Values. 

The goal of SHAP is to explain the prediction of an instance $x$ by computing the contribution of each feature to the prediction.

### TreeSHAP

TreeSHAP is a variant of SHAP for tree-based machine learning models such as decision trees, random forests and gradient boosted trees.

Algorithm:
* If we conditioned on all features – then the prediction from the node in which the instance $x$ falls would be the expected prediction.
* If we did no condition on any feature – we would use the weighted average of predictions of all terminal nodes.
* If use some, but not all, features, we ignore predictions of unreachable nodes. From the remaining terminal nodes, we average the predictions weighted by node sizes (i.e. number of training samples in that node). The mean of the remaining terminal nodes, weighted by the number of instances per node, is the expected prediction for $x$ given.

In [ ]:
import shap

explainer = shap.TreeExplainer(lgb_model, train, model_output="probability")

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(train)
shap.force_plot(explainer.expected_value, shap_values[10,:], train.iloc[10,:])

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[:10], train.iloc[:10,:])

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values[:5, :], feature_names=list(train.columns))

In [ ]:
shap.dependence_plot("Title", shap_values, train)

In [ ]:
shap.summary_plot(shap_values, train)

## Lime (Local Surrogate)

Local surrogate models are interpretable models that are used to explain individual predictions of black box machine learning models. [Local interpretable model-agnostic explanations (LIME)](https://arxiv.org/abs/1602.04938) is a paper in which the authors propose a concrete implementation of local surrogate models. Surrogate models are trained to approximate the predictions of the underlying black box model. Instead of training a global surrogate model, LIME focuses on training local surrogate models to explain individual predictions.

The idea is quite intuitive:
1. You need just train data (without labels). The goal is to understand why the machine learning model made a certain prediction.
2. LIME generates a new dataset consisting of permuted samples and the corresponding predictions of the black box model.
3. On this new dataset LIME then trains an interpretable model (regression or dicision tree), which is weighted by the proximity of the sampled instances to the instance of interest. The learned model should be a good approximation of the machine learning model predictions locally, but it does not have to be a good global approximation.

Mathematically, local surrogate models with interpretability constraint can be expressed as follows:
$$explained = arg min L(f, g, \pi_x) + \Omega(g)$$
The explanation model for instance x is the model $g$ (e.g. linear regression model) that minimizes loss $L$ (e.g. mean squared error), which measures how close the explanation is to the prediction of the original model f (e.g. an xgboost model), while the model complexity $\Omega(g)$ is kept low (e.g. prefer fewer features). 

In [ ]:
from eli5 import show_weights, explain_prediction

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(train, y)

In [ ]:
show_weights(xgb_model)

In [ ]:
explain_prediction(xgb_model, train.iloc[1])

**Main source** – [Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/)

### Homework

1. Use a dataset and a boosting model from the previous homework.
2. Make model interpretation using shap and lime approaches.
3. Make a conclusion based on your research about important features and data dependencies.